In [1]:
import pandas as pd
import numpy as np

!pip install xlrd


[notice] A new release of pip available: 22.3.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
#df = pd.read_excel("C:/User/Desktop/FDM/Assignment 3/Sample - Superstore.xlsx")

df = pd.read_excel("C:/Users/Group1/Downloads/Sample - Superstore.xlsx")


In [20]:

# Create an empty DataFrame to store the results
result_df = pd.DataFrame()

# List of regions and categories
regions = df['Region'].unique()
categories = df['Category'].unique()

# Calculate the total sales and profit for all categories in all regions
total_sales_all_categories = df['Sales'].sum()
total_profit_all_categories = df['Profit'].sum()


# Iterate through regions
for region in regions:
    # Filter the dataset based on the current region
    region_df = df[df['Region'] == region]

    # Calculate the total sales and profit for all categories in the current region
    total_sales_in_region_all_categories = region_df['Sales'].sum()
    total_profit_in_region_all_categories = region_df['Profit'].sum()

    # Iterate through categories in this region
    for category in categories:
        # Filter the dataset based on the current category
        category_df = region_df[region_df['Category'] == category]

        # Calculate the required values
        category_sales = category_df['Sales'].sum()
        category_profit = category_df['Profit'].sum()

        # Calculate the percentages
        sales_in_region_percentage = (category_sales / total_sales_in_region_all_categories) * 100
        sales_in_all_regions_percentage = (category_sales / total_sales_all_categories) * 100
        profit_in_region_percentage = (category_profit / total_profit_in_region_all_categories) * 100
        profit_in_all_regions_percentage = (category_profit / total_profit_all_categories) * 100
        profit_to_sales_ratio_percentage = (category_profit / category_sales) * 100


        # Create a DataFrame for the current row with all necessary columns
        row_data = pd.DataFrame({
            'Region': [region],
            'Category': [category],
            'Sales in Region / Total Sales in Region': [sales_in_region_percentage],
            'Sales in Region / Total Sales in All Regions': [sales_in_all_regions_percentage],
            'Profit in Region / Total Profit in Region': [profit_in_region_percentage],
            'Profit in Region / Total Profit in All Regions': [profit_in_all_regions_percentage],
            'Profit in Region / Sales in Region': [profit_to_sales_ratio_percentage]
        })

        # Concatenate the row to the result DataFrame
        result_df = pd.concat([result_df, row_data], ignore_index=True)

# Print or save the result DataFrame as needed
print(result_df)


     Region         Category  Sales in Region / Total Sales in Region  \
0     South        Furniture                                29.944377   
1     South  Office Supplies                                32.076662   
2     South       Technology                                37.978961   
3     South              NaN                                 0.000000   
4      West        Furniture                                34.821148   
5      West  Office Supplies                                30.443293   
6      West       Technology                                34.735559   
7      West              NaN                                 0.000000   
8   Central        Furniture                                32.678397   
9   Central  Office Supplies                                33.322650   
10  Central       Technology                                33.998952   
11  Central              NaN                                 0.000000   
12     East        Furniture                       

C:\Users\cocko\AppData\Local\Temp\ipykernel_15180\545992196.py:36: RuntimeWarning: invalid value encountered in scalar divide
  profit_to_sales_ratio_percentage = (category_profit / category_sales) * 100
C:\Users\cocko\AppData\Local\Temp\ipykernel_15180\545992196.py:32: RuntimeWarning: invalid value encountered in scalar divide
  sales_in_region_percentage = (category_sales / total_sales_in_region_all_categories) * 100
C:\Users\cocko\AppData\Local\Temp\ipykernel_15180\545992196.py:34: RuntimeWarning: invalid value encountered in scalar divide
  profit_in_region_percentage = (category_profit / total_profit_in_region_all_categories) * 100


In [21]:

# Extract the year and quarter from the 'Order Date' column
df['Year'] = pd.to_datetime(df['Order Date']).dt.year
df['Quarter'] = pd.to_datetime(df['Order Date']).dt.quarter

# List of years in your data
years = df['Year'].unique()
years.sort()

# List of regions and categories
regions = df['Region'].unique()
categories = df['Category'].unique()

# Create an empty DataFrame to store the results
q_result_df = pd.DataFrame()

# Iterate through regions and categories to calculate Q1 sales vs. previous year and Q1 vs. Q2 for each year
for region in regions:
    for category in categories:
        row_data = {'Region': region, 'Category': category}
        for year in years:
            q1_sales = df[(df['Year'] == year) & (df['Quarter'] == 1) & (df['Region'] == region) & (df['Category'] == category)]['Sales'].sum()
            
            if year > min(years):
                prev_year_q1_sales = df[(df['Year'] == year - 1) & (df['Quarter'] == 1) & (df['Region'] == region) & (df['Category'] == category)]['Sales'].sum()
            else:
                prev_year_q1_sales = 0
            
            q2_sales = df[(df['Year'] == year) & (df['Quarter'] == 2) & (df['Region'] == region) & (df['Category'] == category)]['Sales'].sum()
            
            # Create columns for Q1 vs. Previous Year and Q1 vs. Q2 for each year
            row_data[f'Q1 {year} vs. Previous Year'] = q1_sales - prev_year_q1_sales
            row_data[f'Q1 vs. Q2 {year}'] = q2_sales - q1_sales
            
        # Create a DataFrame for the current combination of region and category
        row_df = pd.DataFrame([row_data])
        
        # Concatenate the row to the result DataFrame
        q_result_df = pd.concat([q_result_df, row_df], ignore_index=True)

# Print or save the result DataFrame as needed
print(q_result_df)


     Region         Category  Q1 2018.0 vs. Previous Year  Q1 vs. Q2 2018.0  \
0     South        Furniture                     8375.508        -4224.6820   
1     South  Office Supplies                     3630.055         7980.0520   
2     South       Technology                    32256.636       -25493.4660   
3     South              NaN                        0.000            0.0000   
4      West        Furniture                     5223.295         3597.7360   
5      West  Office Supplies                     7131.852        -1090.9620   
6      West       Technology                     2650.430         8030.9960   
7      West              NaN                        0.000            0.0000   
8   Central        Furniture                     4584.958         3471.6886   
9   Central  Office Supplies                     2546.476         4178.8060   
10  Central       Technology                     1469.248         1155.9680   
11  Central              NaN                        

In [22]:
merged_df = result_df.merge(q_result_df, on=['Region', 'Category'], how='inner')
merged_df

,Region,Category,Sales in Region / Total Sales in Region,Sales in Region / Total Sales in All Regions,Profit in Region / Total Profit in Region,Profit in Region / Total Profit in All Regions,Profit in Region / Sales in Region,Q1 2018.0 vs. Previous Year,Q1 vs. Q2 2018.0,Q1 2019.0 vs. Previous Year,Q1 vs. Q2 2019.0,Q1 2020.0 vs. Previous Year,Q1 vs. Q2 2020.0,Q1 2021.0 vs. Previous Year,Q1 vs. Q2 2021.0,Q1 nan vs. Previous Year,Q1 vs. Q2 nan
0,South,Furniture,29.944377,5.106142,14.484040,2.364273,5.772619,8375.508,-4224.6820,-2130.1720,380.3675,-3226.2860,3193.374,-1395.4200,10581.9265,0.0,0.0
1,South,Office Supplies,32.076662,5.469742,42.752163,6.978562,15.906235,3630.055,7980.0520,6011.1580,-4484.1400,-2413.5090,-2186.833,-3225.0460,5133.3280,0.0,0.0
2,South,Technology,37.978961,6.476207,42.763797,6.980461,13.437908,32256.636,-25493.4660,-31698.7420,3913.5340,13129.0560,-6558.417,-5671.0560,-32.0220,0.0,0.0
3,South,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.0,0.0
4,West,Furniture,34.821148,10.996514,10.611617,4.017133,4.554382,5223.295,3597.7360,5348.3510,-2570.5385,-5451.1760,13338.697,1691.6770,4937.3805,0.0,0.0
5,West,Office Supplies,30.443293,9.613988,48.524812,18.369552,23.821180,7131.852,-1090.9620,-3389.0020,4220.6680,4516.8810,-2647.709,6894.5110,-3248.1560,0.0,0.0
6,West,Technology,34.735559,10.969485,40.863571,15.469312,17.581383,2650.430,8030.9960,6528.1360,1044.4380,1758.2480,4766.348,18017.6560,-7621.3650,0.0,0.0
7,West,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.0,0.0
8,Central,Furniture,32.678397,7.130273,-7.230704,-1.002472,-1.752808,4584.958,3471.6886,1665.5296,462.6954,2971.3634,42.685,1499.4172,358.5948,0.0,0.0
9,Central,Office Supplies,33.322650,7.270846,22.364123,3.100584,5.316512,2546.476,4178.8060,-665.5300,7599.1200,4118.7160,3125.028,11332.1860,-5479.2680,0.0,0.0
